<h1 align="center">Devoir #3</h1>
<h4 align="center">Théo Dionne et Jérôme Leblanc</h4>
<h5 align="center">F I I I I L L L L    O U T T T</h5>

In [ ]:
import numpy as np
from scipy.optimize import brentq
from scipy.integrate import odeint

import matplotlib.pyplot as plt
import methode_tir as mt

# Méthode du tir

D'abord, on cherche à résoudre l'équation de Schrödinger avec la méthode du tir. Le hamiltonien est de la forme d'un oscillateur harmonique, soit:
$$
    {\psi}'' = (x^2-2E)\psi
$$
Comme à l'habitude, pour résoudre cette équation du second ordre, on s'intéresse à un système *réduit* avec le changement de variables typique $\xi \equiv \psi'$:
$$
    \begin{bmatrix}{\psi}'\\{\xi}'\end{bmatrix} = \begin{bmatrix}\xi\\(x^2-2E)\psi\end{bmatrix}
$$
Maintenant, on fixe $\psi(-L)=0$ et ${\psi}'(-L) = 0.001$ pour ensuite trouver les valeurs de $E$ tel que $\psi(L|E)=0$ 

In [ ]:
x_range = np.linspace(-5,5,300)
Schro_fn_E = lambda E : mt.calcul_Schrodinger([0,0.001], x_range, E)[-1,0]
racines = mt.trouver_premieres_racines(Schro_fn_E, 0.4, 6)

for i in range(len(racines)):
    plt.plot(x_range, 5*mt.normaliseur(mt.calcul_Schrodinger([0,0.001], x_range, racines[i])[:,0])+racines[i])
